## Deliverable 2. Create a Customer Travel Destinations Map.

In [44]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [45]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Xinmin,CN,41.9908,122.8253,26.60,86,24,4.47,mist
1,1,Gat,IL,31.6100,34.7642,57.99,100,58,3.27,broken clouds
2,2,Castro,BR,-24.7911,-50.0119,60.55,58,0,3.33,clear sky
3,3,Norman Wells,CA,65.2820,-126.8329,-29.20,68,20,4.61,few clouds
4,4,Nikolskoye,RU,59.7035,30.7861,10.00,78,75,11.18,light snow


In [46]:
# 2. Prompt the user to enter minimum and maximum temperature criteria .
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [47]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]


In [48]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [49]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df= preferred_cities_df.dropna()

In [50]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Takoradi,GH,79.52,overcast clouds,4.8845,-1.7554,
10,Rikitea,PF,78.76,light rain,-23.1203,-134.9692,
14,Pontian Kecil,MY,78.01,few clouds,1.4801,103.3899,
16,Namatanai,PG,82.83,overcast clouds,-3.6667,152.4333,
17,Sao Joao Da Barra,BR,76.12,overcast clouds,-21.6403,-41.0511,
22,Georgetown,MY,81.00,few clouds,5.4112,100.3354,
23,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,
24,Inhambane,MZ,77.90,overcast clouds,-23.8650,35.3833,
29,Atuona,PF,79.45,light rain,-9.8000,-139.0333,
31,Acapulco,MX,84.20,scattered clouds,16.8634,-99.8901,


In [51]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "90.000, 0.000"}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"

    # 6e. Make request and retrieve the JSON data from the search. 
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [52]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Takoradi,GH,79.52,overcast clouds,4.8845,-1.7554,Raybow International Hotel
10,Rikitea,PF,78.76,light rain,-23.1203,-134.9692,Pension Maro'i
14,Pontian Kecil,MY,78.01,few clouds,1.4801,103.3899,SUNFLOWER EXPRESS HOTEL
16,Namatanai,PG,82.83,overcast clouds,-3.6667,152.4333,Namatanai Hotel
17,Sao Joao Da Barra,BR,76.12,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas


In [53]:
# 7. Drop the rows where there is no Hotel Name.
indexNames =hotel_df[hotel_df['Hotel Name'] == ''].index
 
# Delete these row indexes from dataFrame
clean_hotel_df = hotel_df.drop(indexNames)


In [54]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [70]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dt>{Hotel Name}</dt>
<dt><b>City</b></dt><dt>{City}</dt>
<dt><b>Country </b></dt><dt>{Country}</dt>
<dt><b>Current Weather</b></dt><dt>{Current Description} and {Max Temp} °F</dt>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [71]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))